In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
 # Ensure ModernBERT support

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 12.0 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.0.0.dev0 which is incompatible.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.5 which is incompatible.


In [2]:
pip install "transformers>=4.41.0,<5.0.0" "pydantic>=2.0,<2.12"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 125.4 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 38.0 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.5
    Uninstalling pydantic_core-2.41.5:
      Successfully uninstalled pydantic_core-2.41.5
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.5
    Uninstalling pydantic-2.12.5:
      Successfully uninstalled pydantic-2.12.5
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.2.3
    Uninstalling huggingface_hub-1.2.3:
      Successfully uninstalled h

# GoEmotions Dataset Preprocessing for Multi-Label Classification

## 📌 Overview
This notebook prepares the **GoEmotions dataset** for multi-label emotion classification tasks using deep learning models. The dataset consists of Reddit comments annotated with multiple fine-grained emotions.

---

## 📊 Dataset Details
- **Source:** Google Research
- **Dataset:** GoEmotions (Simplified)
- **Total Labels:** 28 (27 emotions + neutral)
- **Task Type:** Multi-label classification
- **Input:** Text (Reddit comments)
- **Output:** Binary emotion vectors

---

## ⚙️ Preprocessing Steps

### 1. Load Dataset
The dataset is loaded using the Hugging Face `datasets` library with the `simplified` configuration to remove rare labels.

### 2. Extract Emotion Labels
All emotion names are extracted from the dataset schema to maintain a consistent label order.

### 3. Create Label Mappings
- `label2id`: Maps emotion names to numeric IDs
- `id2label`: Maps numeric IDs back to emotion names

These mappings are essential for model training and evaluation.

### 4. Multi-Label One-Hot Encoding
Each text sample can express **multiple emotions**.  
To support this, labels are converted into **binary vectors**:

- Vector length = number of emotions
- `1.0` indicates presence of an emotion
- `0.0` indicates absence

#### Example:
Emotion indices: `[2, 5]`

Binary vector:


In [3]:
from datasets import load_dataset
import numpy as np

# Load the dataset
dataset = load_dataset("google-research-datasets/go_emotions", "simplified")

# Get labels
labels = dataset["train"].features["labels"].feature.names
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

def preprocess_data(examples):
    # Convert labels to a binary vector (One-Hot Encoding for Multi-label)
    batch_labels = []
    for labels_list in examples["labels"]:
        binary_labels = [0.0] * len(labels)
        for l in labels_list:
            binary_labels[l] = 1.0
        batch_labels.append(binary_labels)
    
    examples["label_vec"] = batch_labels
    return examples

dataset = dataset.map(preprocess_data, batched=True)

README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

simplified/test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

# Tokenization and Dataset Preparation for Transformer Training

## 📌 Overview
This section tokenizes the GoEmotions dataset using a BERT-compatible tokenizer and prepares it for efficient PyTorch-based training.

---

## 🔤 Tokenizer
- **Model:** ModernBERT-base
- **Tokenizer:** AutoTokenizer
- Ensures vocabulary and token embeddings are aligned with the pretrained model.

---

## ⚙️ Tokenization Strategy

### Configuration
- **Max sequence length:** 128
- **Padding:** Fixed-length (`max_length`)
- **Truncation:** Enabled for long texts

This balances semantic coverage with computational efficiency.

---

## 🔄 Dataset Transformation

### Steps Applied
1. Convert raw text into:
   - `input_ids`
   - `attention_mask`
2. Remove unused columns (`text`, `labels`, `id`)
3. Rename multi-label vector column:
   - `label_vec` → `labels`
4. Convert all features to `torch.Tensor`

---

## ✅ Final Dataset Format
Each training sample contains:
```python
{
  "input_ids": Tensor,
  "attention_mask": Tensor,
  "labels": Tensor  # multi-hot emotion vector
}


In [4]:
from transformers import AutoTokenizer

model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text", "labels", "id"])
tokenized_dataset = tokenized_dataset.rename_column("label_vec", "labels")
tokenized_dataset.set_format("torch")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

# Fine-Tuning ModernBERT for Multi-Label Emotion Classification

## 📌 Overview
This section fine-tunes **ModernBERT-base** on the **GoEmotions dataset** for **multi-label emotion classification** using Hugging Face’s `Trainer` API.

Each input text may express multiple emotions simultaneously, requiring a sigmoid-based output layer and binary cross-entropy loss.

---

## 🧠 Model Configuration
- **Base Model:** ModernBERT-base
- **Architecture:** Encoder-only Transformer
- **Task:** Multi-label classification
- **Number of Labels:** 28 (27 emotions + neutral)
- **Activation:** Sigmoid
- **Loss Function:** Binary Cross-Entropy with Logits

---

## 📊 Evaluation Metrics
To correctly evaluate multi-label predictions, the following metrics are used:

- **Weighted F1 Score:** Handles label imbalance
- **ROC-AUC (Weighted):** Measures ranking quality across all labels

Accuracy is intentionally avoided due to its poor interpretability in multi-label settings.

---

## ⚙️ Training Configuration

| Parameter | Value |
|---------|------|
| Learning Rate | 3e-5 |
| Batch Size | 32 |
| Epochs | 4 |
| Weight Decay | 0.01 |
| Precision | fp16 (mixed) |
| Evaluation | Per epoch |
| Best Model Selection | Weighted F1 |

Additional optimizations:
- Grouping sequences by length to reduce padding
- Mixed precision for faster training and lower memory usage

---

## 🚀 Training Pipeline
1. Tokenize input text using ModernBERT tokenizer
2. Convert emotion labels to multi-hot vectors
3. Fine-tune all model layers end-to-end
4. Evaluate after each epoch
5. Automatically restore best-performing checkpoint

---

## ✅ Outcome
The resulting model learns to predict **multiple fine-grained emotions per text** and is suitable for deployment in:
- Emotion-aware chat systems
- Content moderation pipelines
- Affective computing research

---

## 📦 Deployment Ready
The trained model is fully compatible with:
- Hugging Face Hub
- Transformers inference pipelines
- PyTorch deployment environments


In [9]:
from transformers import ModernBertForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score

model = ModernBertForSequenceClassification.from_pretrained(
    model_id, 
    num_labels=len(labels),
    problem_type="multi_label_classification",
    id2label=id2label,
    label2id=label2id,
    reference_compile=False,   
    attn_implementation="eager" 
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Use torch logic or numpy for sigmoid
    probs = 1 / (1 + np.exp(-logits))
    predictions = (probs > 0.5).astype(float)
    
    # Ensure labels are float for consistency with predictions
    labels = labels.astype(float)
    
    f1_weighted = f1_score(labels, predictions, average="weighted", zero_division=0)
    roc_auc = roc_auc_score(labels, probs, average="weighted")
    
    return {"f1": f1_weighted, "roc_auc": roc_auc}

training_args = TrainingArguments(
    output_dir="./ModernBERT-GoEmotions",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5, 
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True, 
    report_to="none",
    # Added: helps with Kaggle memory management
    dataloader_num_workers=2, 
    group_by_length=True 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_55/805928642.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if

Epoch,Training Loss,Validation Loss,F1,Roc Auc
1,0.090100,0.083718,0.525910,0.911941
2,0.075400,0.080864,0.552808,0.915667
3,0.059100,0.085238,0.567170,0.909428
4,0.039100,0.093749,0.569230,0.902828


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=5428, training_loss=0.06804641310397368, metrics={'train_runtime': 2908.9135, 'train_samples_per_second': 59.692, 'train_steps_per_second': 1.866, 'total_flos': 1.479496440582144e+16, 'train_loss': 0.06804641310397368, 'epoch': 4.0})

In [10]:
model.save_pretrained("./modernbert-goemotions")
tokenizer.save_pretrained("./modernbert-goemotions")


('./modernbert-goemotions/tokenizer_config.json',
 './modernbert-goemotions/special_tokens_map.json',
 './modernbert-goemotions/tokenizer.json')

In [12]:
from huggingface_hub import notebook_login
notebook_login() # Paste your token here

# Push to Hub
model.push_to_hub("Pradeep-mahato/ModernBERT-GoEmotions")
tokenizer.push_to_hub("Pradeep-mahato/ModernBERT-GoEmotions")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Pradeep-mahato/ModernBERT-GoEmotions/commit/701b348eaba392814fb9e4d10193fc3ad3b25dec', commit_message='Upload tokenizer', commit_description='', oid='701b348eaba392814fb9e4d10193fc3ad3b25dec', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Pradeep-mahato/ModernBERT-GoEmotions', endpoint='https://huggingface.co', repo_type='model', repo_id='Pradeep-mahato/ModernBERT-GoEmotions'), pr_revision=None, pr_num=None)

In [13]:
from transformers import pipeline

# 1. Initialize the pipeline using your trained model and tokenizer

pipe = pipeline(
    "text-classification", 
    model=model, 
    tokenizer=tokenizer, 
    top_k=None, # Returns all labels (we will filter them)
    device=0    # Use GPU
)

def predict_emotions(text, threshold=0.4):
    """
    Predicts emotions for a given text and filters by confidence threshold.
    """
    results = pipe(text)[0]
    # Filter results based on the threshold
    emotions = [res for res in results if res['score'] >= threshold]
    
    print(f"\nText: {text}")
    if not emotions:
        print("Result: Neutral/No strong emotion detected.")
    else:
        for e in emotions:
            print(f" - {e['label']}: {e['score']:.4f}")

# 2. Test it out!
test_sentences = [
    "I am so proud of how this model turned out, ModernBERT is amazing!",
    "I'm feeling a bit nervous about the deployment tomorrow.",
    "This is the worst bug I have ever encountered. Why won't it work?"
]

for sentence in test_sentences:
    predict_emotions(sentence)

Device set to use cuda:0



Text: I am so proud of how this model turned out, ModernBERT is amazing!
 - admiration: 0.9882
 - pride: 0.4005

Text: I'm feeling a bit nervous about the deployment tomorrow.
 - nervousness: 0.9413

Text: This is the worst bug I have ever encountered. Why won't it work?
 - disgust: 0.4115
